# Binary file query speed test in Python

Say you want to query the contents of a binary file&mdash;in my case a SIN result file from a SESTRA analysis, but that part is not important. The file has a header with offset tables to tell you the number of rows (records) and the number of columns (fields) for a given matrix (record table). You want to search through the file to find a record that has a certain number in column 4 (index 3). The problem: you cannot guarantee that the matrix will fit in memory.

This notebook creates a dummy binary file with an embedded matrix of floats that does not start at the beginning of the file or end at the end of the file, to make the tests a bit more realistic (similar to the SIN file format, but not with multiple matrices and without the offset tables). In the test case the matrix can be held in memory, it is not enormous, but we want to find a way to extract the relatively few records we are interested in, without holding the whole thing in memory. We also want to do this in Python.

What follows is:

- Creation of the test file along with a verification that it actually works to read it back
- Pure Python tests
- Numpy tests (some hold the whole matrix in memory just to establish a baseline)
- Numpy tests using numexpr to query the matrix. This allows more complicated queries that can still be fast with large matrices

Conclusion: chunked reading with numpy combined with numexpr seems to be a good way forward

In [1]:
import os
import numpy
import struct
import numexpr

In [2]:
def make_data(N, M, filename, header, footer, Nind):
    """
    Make a binary file with an array of records in between a header and a footer
    """
    data = numpy.zeros((N, M), dtype=numpy.float32)
    
    ind = [0] + sorted(numpy.random.randint(0, N, Nind - 2)) + [N-1]
    data[:,0] = numpy.arange(N) + 1
    data[ind,3] = 42.0
    
    with open(filename, 'wb') as f:
        f.write(header.encode('ASCII'))
        data.tofile(f)
        f.write(footer.encode('ASCII'))
    
    return ind

header = 'ABCDEFGHIJ'
footer = 'THEEND!FIN'
N = 1000_0000
M = 10
offset = len(header)
filename = os.path.expanduser('~/DELETEME.bin')


# Make the dummy data
ind0 = make_data(N, M, filename, header, footer, Nind=12)
print(ind0)

def verify(filename, N, M, offset):
    """
    Reade the file just to see that the data is readable
    and that the format is as expected
    """
    with open(filename, 'rb') as f:
        # Read header
        print(f.read(offset))
        
        # Read first record
        rec1 = f.read(M * 4)
        print(struct.unpack('%df' % M, rec1))
        
        # Read second record
        rec2 = f.read(M * 4)
        print(struct.unpack('%df' % M, rec2))
        
        # Read last record
        f.seek(offset + (N - 1) * M * 4)
        recN = f.read(M * 4)
        print(struct.unpack('%df' % M, recN))
        
        # Read footer
        print(f.read(offset))

verify(filename, N, M, offset)

def is_ok(ind):
    diff = numpy.array(ind) - numpy.array(ind0)
    return (diff == 1).all()

[0, 362869, 1111553, 1408003, 3442858, 4693057, 6975679, 8670366, 8711933, 9209097, 9922566, 9999999]
b'ABCDEFGHIJ'
(1.0, 0.0, 0.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(10000000.0, 0.0, 0.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
b'THEEND!FIN'


In [3]:
def read_one_by_one(filename, N, M, offset):
    """
    Pure Python: read one record a a time
    """
    ind = []
    with open(filename, 'rb') as f:
        f.seek(offset)
        for _ in range(N):
            rec = f.read(M * 4)
            values = struct.unpack('%df' % M, rec)
            if values[3] > 0:
                ind.append(int(values[0]))
    assert is_ok(ind)

%timeit -n1 -r3 read_one_by_one(filename, N, M, offset)

4.13 s ± 103 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [4]:
def read_chunked(filename, N, M, offset, chunksize):
    """
    Pure Python: read the file in chunks to reduce the number of read() calls
    """
    ind = []
    with open(filename, 'rb') as f:
        f.seek(offset)
        count = 0
        while count < N:
            Q = min(chunksize, N - count)
            count += chunksize
            data = f.read(Q * M * 4)
            for i in range(Q):
                rec = data[i * M * 4 : (i + 1) * M * 4]
                values = struct.unpack('%df' % M, rec)
                if values[3] > 0:
                    ind.append(int(values[0]))
    assert is_ok(ind)

%timeit -n1 -r3 read_chunked(filename, N, M, offset, 10_000)

5.02 s ± 246 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [5]:
def read_numpy(filename, N, M, offset):
    """
    Read the whole file into a numpy matrix, query in a Python loop
    """
    ind = []
    with open(filename, 'rb') as f:
        f.seek(offset)
        arr = numpy.fromfile(f, dtype=numpy.float32, count=N*M)
        arr = arr.reshape((N, M))
        for values in arr:
            if values[3] > 0:
                ind.append(int(values[0]))
    assert is_ok(ind)

%timeit -n1 -r3 read_numpy(filename, N, M, offset)

13.2 s ± 143 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [6]:
def read_numpy2(filename, N, M, offset):
    """
    Read the whole file into a numpy matrix, query using numpy
    """
    ind = []
    with open(filename, 'rb') as f:
        f.seek(offset)
        arr = numpy.fromfile(f, dtype=numpy.float32, count=N*M)
        arr = arr.reshape((N, M))
        arr2 = arr[arr[:,3] > 0]
        ind = arr2[:,0]
    assert is_ok(ind)

%timeit -n1 -r3 read_numpy2(filename, N, M, offset)

118 ms ± 7.25 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [7]:
def read_numpy_chunked(filename, N, M, offset, chunksize):
    """
    Read the file in chunks using numpy, query using numpy
    """
    ind = []
    with open(filename, 'rb') as f:
        f.seek(offset)
        count = 0
        while count < N:
            Q = min(chunksize, N - count)
            count += chunksize
            arr = numpy.fromfile(f, dtype=numpy.float32, count=Q*M)
            arr = arr.reshape((Q, M))
            sieve = arr[:,3] > 0
            arr2 = arr[sieve]
            ind.extend(arr2[:,0])
    assert is_ok(ind)

%timeit -n1 -r3 read_numpy_chunked(filename, N, M, offset, 10000)

79.6 ms ± 5.95 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [8]:
def read_numpy_chunked_numexpr(filename, N, M, offset, chunksize):
    """
    Read the file in chunks using numpy, query using numexpr
    """
    ind = []
    with open(filename, 'rb') as f:
        f.seek(offset)
        count = 0
        while count < N:
            Q = min(chunksize, N - count)
            count += chunksize
            arr = numpy.fromfile(f, dtype=numpy.float32, count=Q*M)
            arr = arr.reshape((Q, M))
            variables = {'c%d' % i: arr[:,i] for i in range(M)}
            sieve = numexpr.evaluate('c3 > 0', variables)
            arr2 = arr[sieve]
            ind.extend(arr2[:,0])
    assert is_ok(ind)

%timeit -n1 -r3 read_numpy_chunked_numexpr(filename, N, M, offset, 10000)

156 ms ± 3.68 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [9]:
def read_numpy_memmap(filename, N, M, offset):
    """
    Read the file in numpy using memmap to avoid holding the whole file in memory
    Query using a Python loop
    """
    ind = []
    with open(filename, 'rb') as f:
        arr = numpy.memmap(f, dtype=numpy.float32, mode='r', shape=(N, M), offset=offset)
        for values in arr:
            if values[3] > 0:
                ind.append(int(values[0]))
    assert is_ok(ind)

%timeit -n1 -r3 read_numpy_memmap(filename, N, M, offset)

32.7 s ± 2.25 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [10]:
def read_numpy_memmap2(filename, N, M, offset):
    """
    Read the file in numpy using memmap to avoid holding the whole file in memory
    Query using numpy
    """
    ind = []
    with open(filename, 'rb') as f:
        arr = numpy.memmap(f, dtype=numpy.float32, mode='r', shape=(N, M), offset=offset)
        arr2 = arr[arr[:,3] > 0]
        ind = arr2[:,0]
    assert is_ok(ind)

%timeit -n1 -r3 read_numpy_memmap(filename, N, M, offset)

32.5 s ± 567 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
